In [1]:
import pandas as pd

df_dyadic = pd.read_csv("out/result/results_dyadic.csv", index_col=["Subject"])
df_dyadic["Subject"] = df_dyadic.index
df_dyadic

FileNotFoundError: [Errno 2] No such file or directory: 'out/result/results_dyadic.csv'

In [ ]:
# CREATE IDENTIFIERS FROM INDEX

df_dyadic["Dyad"] = df_dyadic.index.map(lambda x: x.split('_')[0])
df_dyadic["Partner"] = df_dyadic.index.map(lambda x: x.split('_')[1])
df_dyadic

In [ ]:
# CREATE GROUPS

conditions = {
    "LipSync": ['6', '7', '8', '9', '11'],
    "FaceTracking": ['1', '2', '3', '4', '10'],
}

df_dyadic.loc[df_dyadic.apply(lambda r: r["Dyad"] in conditions["LipSync"], axis=1), "Group"] = "LipSync"
df_dyadic.loc[df_dyadic.apply(lambda r: r["Dyad"] in conditions["FaceTracking"], axis=1), "Group"] = "FaceTracking"


In [ ]:
group_ls = df_dyadic[df_dyadic["Group"] == "LipSync"]
group_ft = df_dyadic[df_dyadic["Group"] == "FaceTracking"]


In [ ]:

print("Prior knowing Facial Tracking:", group_ft["Prior_Knowing"].std())
print("Prior knowing Facial Simulation:",group_ls["Prior_Knowing"].std())
print("Prior knowing Total:",df_dyadic["Prior_Knowing"].std())

In [ ]:
from scipy.stats import shapiro, skew, kurtosis, levene
import numpy as np
from scipy import stats
import math

from pandas.api.types import is_numeric_dtype

transformation = True

for col in ["IA", "Trust", "Similarity", "Smoothness", "Rapport", "Overlap"]:
    if not is_numeric_dtype(df_dyadic[col]):
        continue
    data = df_dyadic[col]

    print(col)
    data_nona = data[~np.isnan(data)]
    print("Outliers", data_nona[(np.abs(stats.zscore(data_nona)) > 3)])
    print("shapiro", shapiro(data_nona).pvalue)
    print("Skew", skew(data_nona))
    print("kurtosis", kurtosis(data_nona))
    print("levene", levene(df_dyadic[df_dyadic["Group"] == "LipSync"][col], df_dyadic[df_dyadic["Group"] == "FaceTracking"][col]))

    if transformation:
        _min = min(data)
        if _min < 0:
            data += abs(_min)
        # df_dyadic[col] = data.apply(lambda x: np.exp2(x) if not np.isnan(x) else x)
        df_dyadic[col] = np.exp2(data)


        print("\nTransformed:")
        data_nona = data[~np.isnan(data)]
        print("Outliers", data_nona[(np.abs(stats.zscore(data_nona)) > 3)])
        print("shapiro", shapiro(data_nona).pvalue)
        print("Skew", skew(data_nona))
        print("kurtosis", kurtosis(data_nona))
        print("levene", levene(df_dyadic[df_dyadic["Group"] == "LipSync"][col], df_dyadic[df_dyadic["Group"] == "FaceTracking"][col]))
        print("\n==================\n")
        df_dyadic[col] = data





In [ ]:
import pingouin as pg
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

dv ="IA"

def add_effect_size(aov):
    aov['mean_sq'] = aov[:]['sum_sq'] / aov[:]['df']
    aov['eta_sq'] = aov[:-1]['sum_sq'] / sum(aov['sum_sq'])
    aov['omega_sq'] = ((aov[:-1]['sum_sq'] - (aov[:-1]['df'] * aov['mean_sq'][-1]))
                        / (sum(aov['sum_sq'])+aov['mean_sq'][-1]))
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov


model = ols(f"{dv} ~ Group", data=df_dyadic).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
print(add_effect_size(aov_table))

print("\nFacetracking Mean & Std", group_ft[dv].mean(), group_ft[dv].std())
print("Simulated Mean & Std", group_ls[dv].mean(), group_ls[dv].std())

print("\nShapiro of residuals: ", shapiro(model.resid))

posthocs = pg.pairwise_ttests(dv=dv, between='Group', data=df_dyadic, effsize="cohen")
pg.print_table(posthocs)

sns.histplot(model.resid, bins=7)

In [ ]:
aov = pg.anova(data=df_dyadic, dv="IA", between="Group")
pg.print_table(aov)